# Annotating pivots

Using the pigeon library.

In [1]:
%%capture --no-display
from dask.distributed import Client
client = Client(address = 'tcp://192.168.62.207:8786')
client

Client Scheduler: tcp://192.168.62.207:8786 Dashboard: http://192.168.62.207:8787/status,Cluster Workers: 10 Cores: 10 Memory: 673.47 GB


In [2]:
%%time
import dask.bag as db
from takco.util import robust_json_loads_lines
from takco.table import from_tabel
import takco

prefix_header_rules = [
  {'find': 'Precededby ', 'header': 'Preceded by'},
  {'find': 'Succeededby ', 'header': 'Succeeded by'},
  {'find': 'Previousrace:', 'header': 'Previous race'},
  {'find': 'Nextrace:', 'header': 'Next race'}
]

# fnames = 'hdfs://bricks07:9000/user/kruit/wikidump-tables/10*.jsonl'
fnames = 'hdfs://bricks07:9000/user/kruit/tabel/2*-aa'
bag = db.read_text(fnames).map_partitions(robust_json_loads_lines)\
    .map_partitions(lambda ts: list(takco.reshape.restructure(ts, prefix_header_rules=prefix_header_rules)))\
    .map(from_tabel).filter(any).persist()
bag.count().compute()

CPU times: user 6.15 s, sys: 480 ms, total: 6.63 s
Wall time: 14.5 s


38848

In [3]:
import takco
def get_header(df):
    return df.takco.header

from collections import Counter
header_count = Counter(dict(bag.map(get_header).frequencies().compute()))
len(header_count)

17506

In [24]:
from takco import link
from takco.linkedstring import LinkedString

h = ((LinkedString('Prefectures', links=((0, 11, 'http://en.wikipedia.org/wiki/Prefectures_of_Japan?curid=74517'),)), LinkedString('NHK', links=((0, 3, 'http://en.wikipedia.org/wiki/NHK?curid=147782'),)), LinkedString('NNN', links=((0, 3, 'http://en.wikipedia.org/wiki/Nippon_News_Network?curid=1327898'),)), LinkedString('JNN', links=((0, 3, 'http://en.wikipedia.org/wiki/Japan_News_Network?curid=981463'),)), LinkedString('FNN', links=((0, 3, 'http://en.wikipedia.org/wiki/Fuji_News_Network?curid=981656'),)), LinkedString('ANN', links=((0, 3, 'http://en.wikipedia.org/wiki/All-Nippon_News_Network?curid=1327961'),)), LinkedString('TXN', links=((0, 3, 'http://en.wikipedia.org/wiki/TXN?curid=981674'),)), LinkedString('JAITS', links=((0, 5, 'http://en.wikipedia.org/wiki/Japanese_Association_of_Independent_Television_Stations?curid=11744937'),))),)

with hs['AgentLikeHyperlink'] as self:
    
    print(link.get_hrefs(h))
    with self.lookup as l:
        print(l.lookup_title('http://en.wikipedia.org/wiki/Japan?'))
    print(self.lookup.lookup_cells(link.get_hrefs(h)))
    print(list(self.find_pivot_cells(h)))

[[['http://en.wikipedia.org/wiki/Prefectures_of_Japan?curid=74517'], ['http://en.wikipedia.org/wiki/NHK?curid=147782'], ['http://en.wikipedia.org/wiki/Nippon_News_Network?curid=1327898'], ['http://en.wikipedia.org/wiki/Japan_News_Network?curid=981463'], ['http://en.wikipedia.org/wiki/Fuji_News_Network?curid=981656'], ['http://en.wikipedia.org/wiki/All-Nippon_News_Network?curid=1327961'], ['http://en.wikipedia.org/wiki/TXN?curid=981674'], ['http://en.wikipedia.org/wiki/Japanese_Association_of_Independent_Television_Stations?curid=11744937']]]
None
{}
[]


In [ ]:
import pandas as pd

import takco
steps = takco.config.build('step', load=['resources/graphs/wikidata.toml','resources/pipelines/TabEL.toml'])
hs = steps[0]['unpivot_heuristics']
hs = {h.name: h for h in hs}
# display.display(hs)

def find_pivot(headertext):
    return takco.reshape.find_longest_pivot(headertext, hs.values())

import re
for h, count in header_count.most_common():
    if not any(sum(bool(re.match('^[A-Z]{3}$', str(c))) for c in r) > len(r)/2 for r in h):
        continue
#     if c < 10:
#         continue
    if h and all(all(isinstance(c, str) for c in r) for r in h):
        print(h)
        df = pd.DataFrame.takco.from_header(h)
        p = find_pivot(h)
        if p:
            level, colfrom, colto, hname = p
            head, cell = None, None
            for h,c in hs[hname].split_header(h[level], colfrom, colto):
                if h != c:
                    head, cell = h, c
            print(count, p, head, cell)
            display.display(df.takco.highlight_pivot(level, colfrom, colto))
        else:
            print(count)
            display.display(df.takco)

((LinkedString('', links=()), LinkedString('Team', links=()), LinkedString('ARI', links=()), LinkedString('ATL', links=()), LinkedString('CHC', links=()), LinkedString('CIN', links=()), LinkedString('COL', links=()), LinkedString('LAD', links=()), LinkedString('MIA', links=()), LinkedString('MIL', links=()), LinkedString('NYM', links=()), LinkedString('PHI', links=()), LinkedString('PIT', links=()), LinkedString('SD', links=()), LinkedString('SF', links=()), LinkedString('STL', links=()), LinkedString('WSH', links=()), LinkedString('AL', links=())),)
3 (0, 2, 17, 'short-uppercase-acronyms') None None


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
,,Team,ARI,ATL,CHC,CIN,COL,LAD,MIA,MIL,NYM,PHI,PIT,SD,SF,STL,WSH,AL


((LinkedString('Prefectures', links=((0, 11, 'http://en.wikipedia.org/wiki/Prefectures_of_Japan?curid=74517'),)), LinkedString('NHK', links=((0, 3, 'http://en.wikipedia.org/wiki/NHK?curid=147782'),)), LinkedString('NNN', links=((0, 3, 'http://en.wikipedia.org/wiki/Nippon_News_Network?curid=1327898'),)), LinkedString('JNN', links=((0, 3, 'http://en.wikipedia.org/wiki/Japan_News_Network?curid=981463'),)), LinkedString('FNN', links=((0, 3, 'http://en.wikipedia.org/wiki/Fuji_News_Network?curid=981656'),)), LinkedString('ANN', links=((0, 3, 'http://en.wikipedia.org/wiki/All-Nippon_News_Network?curid=1327961'),)), LinkedString('TXN', links=((0, 3, 'http://en.wikipedia.org/wiki/TXN?curid=981674'),)), LinkedString('JAITS', links=((0, 5, 'http://en.wikipedia.org/wiki/Japanese_Association_of_Independent_Television_Stations?curid=11744937'),))),)
3 (0, 1, 6, 'short-uppercase-acronyms') None None


,0,1,2,3,4,5,6,7
,Prefectures,NHK,NNN,JNN,FNN,ANN,TXN,JAITS


((LinkedString('Parliament', links=()), LinkedString('MSP', links=()), LinkedString('MSP', links=()), LinkedString('MSP', links=()), LinkedString('MSP', links=()), LinkedString('MSP', links=()), LinkedString('MSP', links=()), LinkedString('MSP', links=())),)
2 (0, 1, 7, 'short-uppercase-acronyms') None None


...

((LinkedString('Year', links=()), LinkedString('Song', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Chart peak positions', links=()), LinkedString('Album', links=())), (LinkedString('Year', links=()), LinkedString('Song', links=()), LinkedString('UK', links=((0, 2, 'http://en.wikipedia.org/wiki/UK_Singles_Chart?curid=3349146'),)), LinkedString('AUT', links=((0, 3, 'http://en.wikipedia.org/wiki/Ö3_Austria_Top_40?curid=13757814'),)), LinkedString('FRA', links=()), LinkedString('GER', links=()), LinkedString('IRE', links=((0, 3, 'http://en.wikipedia.org/wiki/Irish_Singles_Chart?curid=3627762'),)), LinkedString('SWE', links=()), LinkedString('SWI', links=()), LinkedString('US Dan

,0,1,2,3,4,5,6,7,8,9,10
,Year,Song,Chart peak positions,Chart peak positions,Chart peak positions,Chart peak positions,Chart peak positions,Chart peak positions,Chart peak positions,Chart peak positions,Album
,Year,Song,UK,AUT,FRA,GER,IRE,SWE,SWI,US Dance,Album


((LinkedString('Title', links=()), LinkedString('Album details', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Peak chart positions', links=()), LinkedString('Certifications', links=((0, 14, 'http://en.wikipedia.org/wiki/Music_recording_sales_certification?curid=308242'),))), (LinkedString('Title', links=()), LinkedString('Album details', links=()), LinkedString('US', links=((0, 2, 'http://en.wikipedia.org/wiki/Billboard_200?curid=730350'),)), LinkedString('AUS', links=((0, 3, 'http://en.wikipedia.org/wiki/Australian_Recording_Industry_Association?curid=308882'),)), LinkedString('AU

,0,1,2,3,4,5,6,7,8,9,10,11,12
,Title,Album details,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Peak chart positions,Certifications
,Title,Album details,US,AUS,AUT,FRA,GER,IRE,ITA,JPN,NZ,UK,Certifications


((LinkedString('', links=()), LinkedString('BAR', links=((0, 3, 'http://en.wikipedia.org/wiki/FC_Barcelona?curid=68187'),)), LinkedString('DK', links=((0, 2, 'http://en.wikipedia.org/wiki/FC_Dynamo_Kyiv?curid=889025'),)), LinkedString('INT', links=((0, 3, 'http://en.wikipedia.org/wiki/F.C._Internazionale_Milano?curid=15116'),)), LinkedString('RUB', links=((0, 3, 'http://en.wikipedia.org/wiki/FC_Rubin_Kazan?curid=896135'),))),)
1

In [25]:
def show_pivot(pivot):
    tableHeaders = headers[pivot['headerId']]
    def make_cell(c,ri,ci,pivot):
        cell = c.get("tdHtmlString", "<th>"+c.get('text','')+"</th>")
        if ri == pivot['level'] and ci in range(pivot['colfrom'],pivot['colto']+1):
            cell = cell[:3] + ' style="background:#8c8;"' + cell[4:]
        return cell
    head = [
        [make_cell(c,ri,ci,pivot) for ci,c in enumerate(r)]
        for ri,r in enumerate(tableHeaders)
    ]
    head = [[c.replace("span=", "=") for c in row] for row in head]
    head = "".join(f'<tr>{"".join(row)}</tr>' for row in head)
    html = "<table>" + head + "</table>"
    display.display(display.HTML( html ))
    

import itertools
ns = takco.reshape.NumSuffix()
pivots = takco.reshape.yield_pivots(headers.values(), {'numsuffix':ns})
pivots = itertools.islice(pivots, 50)

from pigeon import annotate
annotations = annotate(
    list(pivots),
    options=['good', 'bad'],
    display_fn = show_pivot
)

HTML(value='0 examples annotated, 51 examples left')

Output()

Annotation done.


In [26]:
annotations

[({'headerId': 7216962826754076956,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId': 3441036834358993182,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId': 3836035899065722383,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 778735725590271279,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId': 2036622794261251368,
   'level': 0,
   'colfrom': 0,
   'colto': 0,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 2873865634436721714,
   'level': 0,
   'colfrom': 4,
   'colto': 14,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 130443667544424743,
   'level': 0,
   'colfrom': 1,
   'colto': 2,
   'heuristic': 'numsuffix'},
  'good'),
 ({'headerId': 5596464666231700013,
   'level': 0,
   'colfrom': 1,
   'colto': 1,
   'heuristic': 'numsuffix'},
  'bad'),
 ({'headerId'